<a href="https://colab.research.google.com/github/dldbwls00/NLP-sentiment-analysis-project/blob/main/kobert_sentiment_0123ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kobert finetuning

In [1]:
# package install
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install tokenizers
!pip install torch
!pip install pandas
!pip install transformers==4.30.0
!pip install gluonnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661774 sha256=c65f883fabce214d054f51b0f660d383ce56daab45e0179a802fd9df87d5d327
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Unin

In [2]:
# kobert install
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-3o7ks4dw/kobert-tokenizer_ad50549eea434d568e93ea9f47731927
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-3o7ks4dw/kobert-tokenizer_ad50549eea434d568e93ea9f47731927
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=0446a7a1c49cffa47091eec47b194ae3d4f2f223b2308b99b8ffedaa588be5ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_t3x9m_/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [4]:
# importing kobert using hugging face
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# gpu setting
device = torch.device("cuda:0")
!nvidia-smi
# device = torch.device("cpu")

Tue Jan 23 11:21:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
# tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [7]:
# connecting google drive - colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# loading dataset
import pandas as pd
conv_training = pd.read_excel('/content/drive/MyDrive/dataset/filtered_corpus_data.xlsx')

In [9]:
print(len(conv_training))
conv_training.sample(n=5)

42874


,emotion,sentences
11180,5,친구들과 주말에 만나서 놀기로 했는데 맘에 드는 옷이 없어서 낙담이 돼. 친구들은 ...
22100,3,고객들 상대할 때가 제일 어려워. 사 갈 때는 이상 없던 옷이 며칠 지나서 망가져...
41132,5,결혼했지만 벌이가 넉넉하지 않아서 저축할 돈이 한 푼도 없으니 참 불안하고 두려워....
9725,5,친구 따위는 세상에 필요 없는 것 같아. 돈이 최고야. 텔레비전이나 뉴스만 봐도 몰...
17708,3,형은 형이고 나는 난 데 부모님은 왜 자꾸 형이랑 나를 비교할까? 맞아. 그럴 때 ...


In [10]:
# 사랑 데이터
conv_love = pd.read_excel('/content/drive/MyDrive/dataset/filtered_love.xlsx')
conv_love.sample(n=5)

,emotion,sentences
1934,1,오늘도 짝사랑하는 남자 땜에 우울해요. 짝사랑의 늪에서 빠져나오세요.
2124,1,재수학원에서 연애는 필망? 목적이 확실한 곳이니까요.
2832,1,친구도 아니고 사귀는 사이도 아닌데 뭐지? 본인의 감정에 귀 기울여보세요.
1279,1,썸 고백해도 될까 당당하게 다가가세요.
1879,1,연애세포 다 죽었나봐 살릴 수 있어요!


In [11]:
# 외로움 데이터
conv_lonely = pd.read_excel('/content/drive/MyDrive/dataset/filtered_lonely.xlsx')
conv_lonely.sample(n=5)

,emotion,sentences
91,6,혼자 사는 일상에서 갑자기 찾아온 외로움. 가족과의 소통이 떨어지면서 나는 더욱 고...
52,6,"사랑이 떠난 뒤, 마음의 무게는 더욱 무거워진다. 그 사람의 존재가 내 생활의 일부..."
215,6,마음이 허전해 널 만나고부터 너무 그런 거 같아
63,6,"외로움은 마치 한숨처럼 깊게 느껴진다. 헤어진 사람과의 기억이 나를 휩쓸면서, 외로..."
161,6,"외로움이 마음을 더욱 예민하게 만들어, 그 감정을 글로 기록하고 싶다."


In [12]:
# concat
conv_training = pd.concat([conv_training, conv_love])
conv_training = pd.concat([conv_training, conv_lonely])
conv_training

,emotion,sentences
0,2,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,2,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,2,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,2,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...
276,6,"경쟁이 아닌 즐거움을 찾는 곳인데, 외로움 속에서는 조금 아쉬워."
277,6,"혼자 배드민턴을 치는 동안, 함께 할 동료가 있다면 더 좋을 것 같아."
278,6,혼자보다는 다른 사람들과 함께 하는 경험이 필요해.
279,6,다같이 하러 왔는데 외로움 때문에 뭔가 씁쓸한 느낌이야.


In [13]:
# extract column -> to array
data_list = []
for q, label in zip(conv_training['sentences'], conv_training['emotion']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [14]:
# array check
print(len(data_list))
print(data_list[0])
print(data_list[1000])
print(data_list[2000])
print(data_list[3000])
print(data_list[4000])
print(data_list[-1])

46118
['일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고. ', '2']
['나는 은퇴할 나이가 되었는데 아내가 작년에 교통사고로 죽으면서 우리 가족의 미래가 불투명한 것 같아. 혼자 일해서 자식들을 키우는 것도 힘들었는데 이제는 정말 대책이 없어. 자식들에게 이 상황을 솔직하게 말하는 게 우선일 것 같아.', '5']
['내가 너무 자주 병원을 가는 것 같아. 가끔이라면 별문제 없지만 자주 병원을 가니까 젊은 시절에 관리를 너무 하지 않은 것 같아. 시간을 내서 운동해야겠어.', '2']
['출산예정일이 지났는데 진통이 오지 않아서 마음이 조마조마해. 태아는 잘 자라고 있다고 했어. 첫 아이는 조금 늦을 수 있다던데 그래서 그런가? 경험하지 못 한 일이라 두렵고 초조해.', '3']
['이번에 새로 들어온 동료가 성격이 괜찮아서 참 다행이야. 성격도 나랑 비슷해서 잘 맞고 일 처리도 마음에 들어. ', '0']
['외로움이 재미를 빼앗아 가는 것 같아서 찝찝하다.', '6']


In [15]:
# splitting data
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

34588
11530


In [16]:
"""class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
"""


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
# preprocessing
tok = tokenizer.tokenize
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [19]:
# kobert training model
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
# load model
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [21]:
# prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# model training
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-22-4b508a570d4d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/541 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.005774736404419 train acc 0.109375


In [ ]:
# test
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
tok = tokenizer.tokenize

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 1:
                test_eval.append("사랑")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("불안")
            elif np.argmax(logits) == 4:
                test_eval.append("상처")
            elif np.argmax(logits) == 5:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 6:
                test_eval.append("외로움")

        print(test_eval[0])

In [ ]:
# test loop: 0 to quit
end = 1
while end == 1 :
    sentence = input("write sentences: ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")